In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import simpy as sp
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [2]:
from sympy import symbols, Matrix
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy import simplify, cos, sin, atan2, asin, exp, tan

def dir_cosine(q):
        return Matrix([
            [1 - 2 * (q[2] ** 2 + q[3] ** 2), 2 * (q[1] * q[2] +
                                                   q[0] * q[3]), 2 * (q[1] * q[3] - q[0] * q[2])],
            [2 * (q[1] * q[2] - q[0] * q[3]), 1 - 2 *
             (q[1] ** 2 + q[3] ** 2), 2 * (q[2] * q[3] + q[0] * q[1])],
            [2 * (q[1] * q[3] + q[0] * q[2]), 2 * (q[2] * q[3] -
                                                   q[0] * q[1]), 1 - 2 * (q[1] ** 2 + q[2] ** 2)]
        ])

def omega(w):
        return Matrix([
            [0, -w[0], -w[1], -w[2]],
            [w[0], 0, w[2], -w[1]],
            [w[1], -w[2], 0, w[0]],
            [w[2], w[1], -w[0], 0],
        ])

def skew(v):
    return Matrix([
            [0, -v[2], v[1]],
            [v[2], 0, -v[0]],
            [-v[1], v[0], 0]
        ])

def get_R_from_euler(e) :
    phi = e[0]
    theta = e[1]
    psi = e[2]

    C_B_I = zeros(3,3)
    C_B_I[0,0] = cos(psi) * cos(theta)
    C_B_I[0,1] = sin(psi) * cos(theta)
    C_B_I[0,2] = -sin(theta)
    C_B_I[1,0] = -sin(psi)*cos(phi) + cos(psi)*sin(theta)*sin(phi)
    C_B_I[1,1] = cos(psi)*cos(phi) + sin(psi)*sin(theta)*sin(phi)
    C_B_I[1,2] = cos(theta)*sin(phi)
    C_B_I[2,0] = sin(psi)*sin(phi) + cos(psi)*sin(theta)*cos(phi)
    C_B_I[2,1] = -cos(psi)*sin(phi)+sin(psi)*sin(theta)*cos(phi)
    C_B_I[2,2] = cos(theta)*cos(phi)

    return C_B_I

In [3]:
J,m = symbols('J, m',positive=True)
x = Matrix(symbols(
    'X Y Z U V W phi theta psi p q r', real=True))
u = Matrix(symbols('T1 T2 T3 M1 M2 M3', real=True))
f = zeros(12, 1)
ix = 12
iu = 6

In [4]:
X = x[0,0]
Y = x[1,0]
Z = x[2,0]

U = x[3,0]
V = x[4,0]
W = x[5,0]

phi = x[6,0]
theta = x[7,0]
psi = x[8,0]

p = x[9,0]
q = x[10,0]
r = x[11,0]

T1 = u[0,0]
T2 = u[1,0]
T3 = u[2,0]
M1 = u[3,0]
M2 = u[4,0]
M3 = u[5,0]

In [5]:
# R_BI = dir_cosine(x[6:10])
R_BI = get_R_from_euler(x[6:9])
print(R_BI)

Matrix([[cos(psi)*cos(theta), sin(psi)*cos(theta), -sin(theta)], [sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), sin(phi)*cos(theta)], [sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi), cos(phi)*cos(theta)]])


In [20]:
f[0:3, 0] = x[3:6,0]
f[3:6, 0] = 1/m*u[0:3,0]
f[6,0] = p + sin(phi) * tan(theta) * q + cos(phi) * tan(theta) * r
f[7,0] = cos(phi) * q - sin(phi) * r
f[8,0] = sin(phi) / cos(theta) * q + cos(phi) / cos(theta) * r
f[9:,0] = 1/J * (u[3:,0] - skew(x[9:,0])*(J*x[9:,0]))

In [21]:
f

Matrix([
[                                                U],
[                                                V],
[                                                W],
[                                             T1/m],
[                                             T2/m],
[                                             T3/m],
[p + q*sin(phi)*tan(theta) + r*cos(phi)*tan(theta)],
[                          q*cos(phi) - r*sin(phi)],
[    q*sin(phi)/cos(theta) + r*cos(phi)/cos(theta)],
[                                             M1/J],
[                                             M2/J],
[                                             M3/J]])

In [18]:
skew(x[9:,0])*x[9:,0]

Matrix([
[0],
[0],
[0]])

In [16]:
J*x[9:,0]

Matrix([
[J*p],
[J*q],
[J*r]])

In [14]:
skew(x[9:,0])*(J*x[9:,0])

Matrix([
[0],
[0],
[0]])

In [19]:
u[4:,0]

Matrix([
[M2],
[M3]])